# Data Preprocessing

In [2]:
import numpy as np
import pandas as pd
import email
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

data_dir="./trec05p-1/"
full_index_dir="./trec05p-1/full/index"

full_index=pd.read_csv(full_index_dir,header=None,names=["label","index"],sep=" ")
full_index["index"]=full_index["index"].apply(lambda x:data_dir+x[3:])
full_index.tail()

,label,index
92184,spam,./trec05p-1/data/307/086
92185,spam,./trec05p-1/data/307/087
92186,spam,./trec05p-1/data/307/088
92187,spam,./trec05p-1/data/307/089
92188,spam,./trec05p-1/data/307/090


In [2]:
Errorlst=[]
EmailInfo=pd.DataFrame({},columns=["email_id","parts","attachments","html","subject","body","links","spam"])

for i,row in full_index.iterrows():

    try:
        with open(row["index"],"r") as f:
            raw_email=f.read()
    except:
        Errorlst.append((i,row,"UnicodeDecodeError"))
        continue

    parsed_email = email.message_from_string(raw_email)
    attachments=0
    parts=0
    html=0

    try:
        if parsed_email.is_multipart():
            for part in parsed_email.walk():
                ctype = part.get_content_type()
                cdispo = str(part.get('Content-Disposition'))
                
                parts+=1

                if 'attachment' in cdispo:
                    attachments+=1
                    
                if ctype == 'text/html':
                    html=1

                # skip any text/plain (txt) attachments
                if (ctype == 'text/plain') and 'attachment' not in cdispo:
                    body = part.get_payload(decode=True)  # decode

        # not multipart - i.e. plain text, no attachments, keeping fingers crossed
        else:
            body = parsed_email.get_payload(decode=True)
            ctype = parsed_email.get_content_type()
            cdispo = str(parsed_email.get('Content-Disposition'))
            
            if 'attachment' in cdispo:
                attachments+=1
            
            if ctype == 'text/html':
                html=1
        
        subject=parsed_email["Subject"]
        
    except:
        Errorlst.append((i,row,"binascii.Error"))
        continue

    soup = BeautifulSoup(body)
    
    email_id = row["index"][-7:]
    label = 1 if row["label"]=="spam" else 0
    links=len(soup.find_all("a"))
    
    Info=pd.DataFrame([[email_id,parts,attachments,html,subject,body,links,label]],columns=EmailInfo.columns)
    EmailInfo=pd.concat([EmailInfo,Info],axis=0,ignore_index=True)
    
    if i%10000==0:
        print("{} done.".format(i))
        
EmailInfo.tail(10)
    

0 done.
10000 done.
20000 done.
30000 done.
50000 done.
60000 done.
70000 done.
80000 done.
90000 done.


,email_id,parts,attachments,html,subject,body,links,spam
85193,307/080,2,0,1,100's of titles by XP Pro Windows XP & Photosh...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85194,307/082,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85195,307/083,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85196,307/084,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85197,307/085,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85198,307/086,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85199,307/087,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85200,307/088,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85201,307/089,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1
85202,307/090,3,0,1,=?gb2312?B?c2hvZXMgZnJvbSB3d3cubG92ZWluZmFzaGl...,"b'www.loveinfashion.com\r\nusername=customer,p...",0,1


In [3]:
EmailInfo[["parts","attachments","html","links","spam"]]=\
EmailInfo[["parts","attachments","html","links","spam"]].astype("int32")
i,EmailInfo.dtypes, EmailInfo.describe()

(92188,
 email_id       object
 parts           int32
 attachments     int32
 html            int32
 subject        object
 body           object
 links           int32
 spam            int32
 dtype: object,
               parts   attachments          html         links          spam
 count  85203.000000  85203.000000  85203.000000  85203.000000  85203.000000
 mean       0.814842      0.028086      0.315411      0.497060      0.552704
 std        1.480620      0.248603      0.464682      4.398732      0.497217
 min        0.000000      0.000000      0.000000      0.000000      0.000000
 25%        0.000000      0.000000      0.000000      0.000000      0.000000
 50%        0.000000      0.000000      0.000000      0.000000      1.000000
 75%        2.000000      0.000000      1.000000      0.000000      1.000000
 max       53.000000     16.000000      1.000000    175.000000      1.000000)

In [4]:
# EmailInfo.to_csv("email_05.csv")

In [5]:
len(Errorlst)

6986

# Structural Information

In [3]:
EmailInfo=pd.read_csv("email_05.csv")

StructuralInfo=pd.DataFrame({"multipart":EmailInfo["parts"],"html":EmailInfo["html"],\
                             "links":EmailInfo["links"],"attachments":EmailInfo["attachments"],\
                             "spam":EmailInfo["spam"]})
StructuralInfo["multipart"]=(StructuralInfo["multipart"]>0).astype(int)
StructuralInfo["links"]=(StructuralInfo["links"]>0).astype(int)
StructuralInfo["attachments"]=(StructuralInfo["attachments"]>0).astype(int)

StructuralInfo.describe()

,multipart,html,links,attachments,spam
count,85203.000000,85203.000000,85203.000000,85203.000000,85203.000000
mean,0.290060,0.315411,0.091030,0.020833,0.552704
std,0.453793,0.464682,0.287653,0.142825,0.497217
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# StructuralInfo.to_csv("email_05_structure.csv")

# Subject Feature

In [1]:
import numpy as np
import pandas as pd
import pickle

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import sys 
sys.path.append("../../")
from E4525_ML import text

import warnings
warnings.filterwarnings("ignore")

EmailInfo=pd.read_csv("email_05.csv")
Subject_content=EmailInfo[["subject","spam"]]
Subject_content.fillna(value="",inplace=True)

In [2]:
stop_words=list(text.stop_words())
stop_words

['.',
 "'re",
 'of',
 'to',
 'could',
 'y',
 'them',
 'who',
 'here',
 'must',
 'been',
 'am',
 '#',
 'ha',
 'into',
 'now',
 '/',
 '^',
 'own',
 'how',
 'from',
 'needn',
 'those',
 'just',
 'between',
 'aren',
 'my',
 'himself',
 'do',
 'so',
 '%',
 'befor',
 'hadn',
 'she',
 'the',
 'whi',
 'what',
 'doe',
 '@',
 'few',
 'hasn',
 'thi',
 "'s",
 'were',
 'if',
 '_',
 'onc',
 's',
 '(',
 'off',
 'after',
 'as',
 'again',
 'most',
 'won',
 ';',
 'would',
 'up',
 'becaus',
 'will',
 'myself',
 'be',
 'm',
 'than',
 'down',
 'other',
 'while',
 'our',
 'it',
 'abov',
 'sha',
 'he',
 'where',
 'through',
 'out',
 're',
 'each',
 'you',
 '~',
 '=',
 '``',
 'i',
 'yourselv',
 'and',
 'itself',
 'in',
 'wo',
 'haven',
 'weren',
 '-',
 'we',
 'over',
 'more',
 '[',
 'there',
 'are',
 'further',
 '*',
 'ain',
 'that',
 'mustn',
 ')',
 'same',
 'against',
 'wasn',
 'their',
 'under',
 'too',
 '>',
 'not',
 'veri',
 'nor',
 "'d",
 '<',
 'hi',
 'then',
 'for',
 '{',
 'no',
 'your',
 'herself',
 '

In [3]:
tokenizer=text.stem_tokenizer
setVectorizer=CountVectorizer(input="content",binary=True,tokenizer=tokenizer,stop_words=stop_words)
X_set=setVectorizer.fit_transform(Subject_content["subject"].iloc[:])
print(X_set.shape,Subject_content["subject"].shape)

(85203, 36750) (85203,)


In [4]:
data_dir = "subject_feature"

set_vectorizer_filename=   data_dir+"/set_vectorizer.p"
set_features_filename=     data_dir+"/set_features.p"
labels_filename=     data_dir+"/labels.p"

pickle.dump(setVectorizer, open( set_vectorizer_filename, "wb" ) )
pickle.dump(X_set,         open( set_features_filename, "wb" ) )
pickle.dump(Subject_content["spam"].values,         open( labels_filename, "wb" ) )

In [5]:
count_Vectorizer=CountVectorizer(input="content",binary=False,tokenizer=tokenizer,stop_words=stop_words)
X_count=count_Vectorizer.fit_transform(Subject_content["subject"])

data_dir = "subject_feature"

count_vectorizer_filename=   data_dir+"/count_vectorizer.p"
count_features_filename=     data_dir+"/count_features.p"

pickle.dump(count_Vectorizer, open( count_vectorizer_filename, "wb" ) )
pickle.dump(X_count,             open( count_features_filename, "wb" ) )

In [6]:
tfidfVectorizer=TfidfVectorizer(input="content",tokenizer=tokenizer,stop_words=stop_words)
X_tfidf=tfidfVectorizer.fit_transform(Subject_content["subject"])

data_dir = "subject_feature"

tfidf_vectorizer_filename=   data_dir+"/tfidf_vectorizer.p"
tfidf_features_filename=     data_dir+"/tfidf_features.p"

pickle.dump(tfidfVectorizer, open( tfidf_vectorizer_filename, "wb" ) )
pickle.dump(X_tfidf,             open( tfidf_features_filename, "wb" ) )

# Message Feature

In [1]:
import numpy as np
import pandas as pd
import pickle
import datetime
import time

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import sys 
sys.path.append("../../")
from E4525_ML import text

import warnings
warnings.filterwarnings("ignore")

# helper routing to log a message with time
def log_message( label_string ):
    ts = time.time()
    st = datetime.datetime.fromtimestamp( ts ).strftime( '%Y-%m-%d %H:%M:%S:%f' )
    print("{}: {}".format(st, label_string))
    with open("data_preprocessing_log.txt","a") as f:
        f.write("{}: {}\n".format(st, label_string))

def extract_text_feature(col="body", data_dir = "message_feature", feature="set",n=-1):
    
    EmailInfo=pd.read_csv("email_05.csv")
    Body_content=EmailInfo[[col,"spam"]]
    Body_content.dropna(axis=0,inplace=True)
    
    stop_words=list(text.stop_words())
    tokenizer=text.stem_tokenizer
    
    if feature=="set":
        setVectorizer=CountVectorizer(input="content",binary=True,tokenizer=tokenizer,stop_words=stop_words)
    elif feature=="count":
        setVectorizer=CountVectorizer(input="content",binary=False,tokenizer=tokenizer,stop_words=stop_words)
    else:
        setVectorizer=TfidfVectorizer(input="content",binary=False,tokenizer=tokenizer,stop_words=stop_words)
    
    log_message( "Start to extract {} feature.".format(feature) )
    if n==-1:
        X_set=setVectorizer.fit_transform(Body_content[col].iloc[:])
    else:
        X_set=setVectorizer.fit_transform(Body_content[col].iloc[:n])
    log_message( "Finished to extract {} feature.".format(feature) )
    
    set_vectorizer_filename=   data_dir+"/{}_vectorizer_{}.p".format(feature,n)
    set_features_filename=     data_dir+"/{}_features_{}.p".format(feature,n)
    labels_filename=     data_dir+"/labels_{}.p".format(n)

    pickle.dump(setVectorizer, open( set_vectorizer_filename, "wb" ) )
    pickle.dump(X_set, open( set_features_filename, "wb" ) )
    pickle.dump(Body_content["spam"].values, open( labels_filename, "wb" ) )
    
    return

%time extract_text_feature(feature="count",n=-1)

2020-04-08 16:55:34:715321: Start to extract count feature.
2020-04-08 22:28:04:589576: Finished to extract count feature.
Wall time: 5h 32min 34s
